In [1]:
### Setup directory and env vars
%cd /home/viresh16118/repos/jodie/
%load_ext dotenv
%dotenv

/home/viresh16118/repos/jodie


In [2]:
### Essential Imports
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from collections import defaultdict
import dyna_loader as dnldr
import os
from tqdm import tqdm, trange, tqdm_notebook, tnrange
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

try:
    get_ipython
    trange = tnrange
    tqdm = tqdm_notebook
except NameError:
    pass

In [3]:
### Load Network
[user2id, user_sequence_id, 
 user_timedifference_sequence, 
 user_previous_itemid_sequence, 
 op_sequence_id, 
 op_timedifference_sequence, 
 item2id, item_sequence_id, 
 item_timedifference_sequence, 
 timestamp_sequence, 
 feature_sequence, 
 y_true_labels] = dnldr.load_network()

Loading network from /home/viresh16118/ICDM_Computed/stackoverflow/feature_extracted.pkl and /home/viresh16118/ICDM_Computed/stackoverflow/id_to_body.pkl
Formating item sequence
Formating user sequence
Formatting original poster sequence
Scaling Timestamps
*** Network loading completed ***




In [4]:
### Inspect Features and initialise useful constants
num_interactions = len(user_sequence_id)
num_users = len(user2id) 
num_items = len(item2id) + 1 # one extra item for "none-of-these"
num_features = len(feature_sequence[0])
print("*** Network statistics:\n  %d users\n  %d items\n  %d interactions\n  %d feature size ***\n\n" % (num_users, num_items, num_interactions, num_features))

*** Network statistics:
  90182 users
  19911 items
  1397086 interactions
  73 feature size ***




In [5]:
### Define train-test and batching parameters
train_proportion = 0.8
train_end_idx = validation_start_idx = int(num_interactions * train_proportion) 
test_start_idx = int(num_interactions * (train_proportion+0.1))
test_end_idx = int(num_interactions * (train_proportion+0.2))

timespan = timestamp_sequence[-1] - timestamp_sequence[0]
tbatch_timespan = timespan / 500 
print(tbatch_timespan)

5355.894


In [6]:
### initialize module style dict for storing tbatch
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

lib = dotdict({
    # list of users of each tbatch up to now
    'current_tbatches_interactionids': defaultdict(list),
    'current_tbatches_user': defaultdict(list),
    'current_tbatches_item': defaultdict(list),
    'current_tbatches_op': defaultdict(list),
    'current_tbatches_timestamp': defaultdict(list),
    'current_tbatches_feature': defaultdict(list),
    'current_tbatches_label': defaultdict(list),
    'current_tbatches_previous_item': defaultdict(list),
    'current_tbatches_user_timediffs': defaultdict(list),
    'current_tbatches_item_timediffs': defaultdict(list),
    'current_tbatches_user_timediffs_next': defaultdict(list),
    
    # the latest tbatch a user is in
    'tbatchid_user': defaultdict(lambda: -1),

    # the latest tbatch a item is in
    'tbatchid_item': defaultdict(lambda: -1),
    
    'total_reinitialization_count': 0,
})
print(lib.tbatchid_user)

defaultdict(<function <lambda> at 0x7f9974bc0620>, {})


In [7]:
### Some auxilary functions
def reinitialize_tbatches(lib):
    # list of users of each tbatch up to now
    lib.current_tbatches_interactionids = defaultdict(list)
    lib.current_tbatches_user = defaultdict(list)
    lib.current_tbatches_item = defaultdict(list)
    lib.current_tbatches_op = defaultdict(list)
    lib.current_tbatches_timestamp = defaultdict(list)
    lib.current_tbatches_feature = defaultdict(list)
    lib.current_tbatches_label = defaultdict(list)
    lib.current_tbatches_previous_item = defaultdict(list)
    lib.current_tbatches_user_timediffs = defaultdict(list)
    lib.current_tbatches_item_timediffs = defaultdict(list)
    lib.current_tbatches_user_timediffs_next = defaultdict(list)

    # the latest tbatch a user is in
    lib.tbatchid_user = defaultdict(lambda: -1)

    # the latest tbatch a item is in
    lib.tbatchid_item = defaultdict(lambda: -1)

    lib.total_reinitialization_count +=1


## Define DynaEmbed Main Model

In [8]:
class CustomCell(nn.Module):
    def __init__(self, embedding_size, feature_size):
        super(CustomCell, self).__init__()
        self.embsz = embedding_size
        self.uembsz = embedding_size
        self.iembsz = embedding_size
        self.featsz = feature_size
        self.N = 3*embedding_size + feature_size + 1
        self.WdynU1 = torch.nn.Parameter(torch.randn(embedding_size, self.uembsz, self.N))
        self.WdynU2 = torch.nn.Parameter(torch.randn(embedding_size, self.uembsz, self.N))
        self.WdynI1 = torch.nn.Parameter(torch.randn(embedding_size, self.iembsz, self.N))
        self.WdynF  = torch.nn.Parameter(torch.randn(embedding_size, self.featsz, self.N))
        self.WdynT  = torch.nn.Parameter(torch.randn(embedding_size, 1, self.N))
        
        ### this parameter remains unused in ItemRNN. Wastes GPU Space.
        ### Probably find some better place for this?
        self.WdynP = torch.nn.Parameter(torch.randn(embedding_size, self.uembsz, self.N))
    
    def forward(self, U1, U2, I1, F, T):
        ntensor = torch.cat([U1, U2, I1, F, T]).unsqueeze(1)
        W1 = torch.mm(self.WdynU1.view(self.embsz * self.uembsz, -1), ntensor).view(self.embsz, self.uembsz, -1)
        W2 = torch.mm(self.WdynU2.view(self.embsz * self.uembsz, -1), ntensor).view(self.embsz, self.uembsz, -1)
        W3 = torch.mm(self.WdynI1.view(self.embsz * self.iembsz, -1), ntensor).view(self.embsz, self.iembsz, -1)
        W4 = torch.mm(self.WdynF.view(self.embsz * self.featsz, -1), ntensor).view(self.embsz, self.featsz, -1)
        W5 = torch.mm(self.WdynT.view(self.embsz * 1, -1), ntensor).view(self.embsz, 1, -1)

        ### All N_i's are of same size
        N_1 = torch.tanh(torch.mm(W1.squeeze(2), U1.unsqueeze(1)))
        N_2 = torch.tanh(torch.mm(W2.squeeze(2), U2.unsqueeze(1)))
        N_3 = torch.tanh(torch.mm(W3.squeeze(2), I1.unsqueeze(1)))
        N_4 = torch.tanh(torch.mm(W4.squeeze(2), F.unsqueeze(1)))
        N_5 = torch.tanh(torch.mm(W5.squeeze(2), T.unsqueeze(1)))
        
        ### Perform predictions over this interaction
        user_predicted, item_predicted = self.predict_(U1, U2, I1, F, T)
        
        # return the sum of all N_i's as new embedding
        return torch.sum(torch.stack([N_1, N_2, N_3, N_4, N_5], dim=1), dim=1), user_predicted, item_predicted
    
    def project_user(self, U1_, U2_, I1_, F_, delta):
        # Predict embedding of user U1 from it's embedding at U_ at T-
        ntensor = torch.cat([U1_, U2_, I1_, F_, delta]).unsqueeze(1)
        W5 = torch.mm(self.WdynT.view(self.embsz * 1, -1), ntensor).view(self.embsz, 1, -1)
        N_5 = torch.tanh(torch.mm(W5.squeeze(2), delta.unsqueeze(1)))
        predicted_u = (1. + N_5).squeeze(1) * U1_
        return predicted_u
    
    def predict_(self, U1_, U2_, I1_, F_, delta):
        user_hat = self.project_user(U1_, U2_, I1_, F_, delta)
        ntensor = torch.cat([U1_, U2_, I1_, F_, delta]).unsqueeze(1)
        W6 = torch.mm(self.WdynP.view(self.embsz * self.uembsz, -1), ntensor).view(self.embsz, self.uembsz, -1)
        item_hat = torch.tanh(torch.mm(W6.squeeze(2), user_hat.unsqueeze(1)))
        return user_hat, item_hat
    

class Wrapper(nn.Module):
    def __init__(self, emb_sz, feat_sz):
        super(Wrapper, self).__init__()
        self.cell = CustomCell(emb_sz, feat_sz)
    
    def forward(self, BU1, BU2, BI1, BF, BT):
        ### Input is batched. Unroll the TBatch and train for every timestep
        output = []
        user_predicted = []
        item_predicted = []
        for ue, oe, ie, fv, delta in zip(torch.unbind(BU1, 0), torch.unbind(BU2, 0), torch.unbind(BI1, 0), torch.unbind(BF, 0), torch.unbind(BT, 0)):
            new_embedding, u_predicted, i_predicted = self.cell(ue, oe, ie, fv, delta)
            output.append(new_embedding)
            user_predicted.append(u_predicted)
            item_predicted.append(i_predicted)
        output = torch.stack(output, dim=0).squeeze()
        user_predicted = torch.stack(user_predicted, dim=0).squeeze()
        item_predicted = torch.stack(item_predicted, dim=0).squeeze()
        return output, user_predicted, item_predicted

class DynaEmbedModel(nn.Module):
    def __init__(self, emb_sz, feat_sz):
        super(DynaEmbedModel, self).__init__()
        print("*** Initializing the DYNAEMBED model ***")
        self.userrnn = Wrapper(emb_sz, feat_sz)
        self.itemrnn = Wrapper(emb_sz, feat_sz)
        print("*** DYNAEMBED initialization complete ***\n\n")
    
    def forward(self, BU1, BU2, BI1, BF, BT, perform='predict'):
        # return new user, item and other user embedding
        if perform=='interacting_user':
            BU1_new, BU1_predicted, I1_predicted = self.userrnn(BU1, BU2, BI1, BF, BT)
            return BU1_new
        elif perform == 'interacted_user':
            BU2_new, _, __ = self.userrnn(BU2, BU1, BI1, BF, BT)
            return BU2_new
        elif perform == 'item':
            _, __, BI1_new = self.itemrnn(BU1, BU2, BI1, BF, BT)
            return BI1_new
        elif perform=='predict':
            _, BU1_predicted, I1_predicted = self.userrnn(BU1, BU2, BI1, BF, BT)
            return BU1_predicted, I1_predicted
print("***Dynaembed Model Defined and available for instantiation***")

***Dynaembed Model Defined and available for instantiation***


In [9]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print("Using CUDA? ==>", use_cuda, device)

### Initialize model with embedding size and stuff
embedding_dim = 128

### the following can be changed for different experiments
initial_userembeddings = F.normalize(torch.randn(embedding_dim), dim=0).to(device)
initial_itemembeddings = F.normalize(torch.randn(embedding_dim), dim=0).to(device)

### with repeat, the all users start from same embedding, and all items start from same embedding
user_embeddings = initial_userembeddings.repeat(num_users, 1)
item_embeddings = initial_userembeddings.repeat(num_items, 1)

learning_rate = 1e-3

model = DynaEmbedModel(embedding_dim, num_features).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

MSELoss = torch.nn.MSELoss()

total_loss, loss, total_interaction_count = 0, 0, 0

Using CUDA? ==> True cuda
*** Initializing the DYNAEMBED model ***
*** DYNAEMBED initialization complete ***




## Training Loop

In [10]:
tbatch_start_time = None
tbatch_to_insert = -1
tbatch_full = False

with trange(train_end_idx) as pbar2:
    for j in pbar2:
        pbar2.set_description("Processed {}th transaction".format(j))
        # READ INTERACTION J
        userid = user_sequence_id[j]
        original_poster = op_sequence_id[j]
        itemid = item_sequence_id[j]
        feature = feature_sequence[j]
        user_timediff = user_timedifference_sequence[j]
        item_timediff = item_timedifference_sequence[j]

        # CREATE T-BATCHES: ADD INTERACTION J TO THE CORRECT T-BATCH
        tbatch_to_insert = max(lib.tbatchid_user[userid], lib.tbatchid_item[itemid]) + 1 
        lib.tbatchid_user[userid] = tbatch_to_insert 
        lib.tbatchid_item[itemid] = tbatch_to_insert

        lib.current_tbatches_user[tbatch_to_insert].append(userid)
        lib.current_tbatches_item[tbatch_to_insert].append(itemid)
        lib.current_tbatches_op[tbatch_to_insert].append(original_poster)
        lib.current_tbatches_feature[tbatch_to_insert].append(feature)
        lib.current_tbatches_interactionids[tbatch_to_insert].append(j)
        lib.current_tbatches_user_timediffs[tbatch_to_insert].append(user_timediff)
        lib.current_tbatches_item_timediffs[tbatch_to_insert].append(item_timediff)
        lib.current_tbatches_previous_item[tbatch_to_insert].append(user_previous_itemid_sequence[j])

        timestamp = timestamp_sequence[j]
        if tbatch_start_time is None:
            tbatch_start_time = timestamp

        # AFTER ALL INTERACTIONS IN THE TIMESPAN ARE CONVERTED TO T-BATCHES, FORWARD PASS TO CREATE EMBEDDING TRAJECTORIES AND CALCULATE PREDICTION LOSS
        if timestamp - tbatch_start_time > tbatch_timespan:
            tbatch_start_time = timestamp # RESET START TIME FOR THE NEXT TBATCHES

            # ITERATE OVER ALL T-BATCHES
            with trange(len(lib.current_tbatches_user)) as pbar3:
                for i in pbar3:
                    pbar3.set_description("Processed {} of {} T-batches".format(i, len(lib.current_tbatches_user)))
#                     print("Processing T-Batch {}/{}".format(i, len(lib.current_tbatches_user)))

                    total_interaction_count += len(lib.current_tbatches_interactionids[i])

                    # LOAD THE CURRENT TBATCH
                    tbatch_userids = torch.LongTensor(lib.current_tbatches_user[i]).to(device) # Recall "lib.current_tbatches_user[i]" has unique elements
                    tbatch_itemids = torch.LongTensor(lib.current_tbatches_item[i]).to(device) # Recall "lib.current_tbatches_item[i]" has unique elements
                    tbatch_opids = torch.LongTensor(lib.current_tbatches_op[i]).to(device)     # original posters may or may not be unique
                    tbatch_interactionids = torch.LongTensor(lib.current_tbatches_interactionids[i]).to(device) 

                    feature_tensor = torch.Tensor(lib.current_tbatches_feature[i]).to(device)
                    user_timediffs_tensor = torch.Tensor(lib.current_tbatches_user_timediffs[i]).to(device).unsqueeze(1)
                    item_timediffs_tensor = torch.Tensor(lib.current_tbatches_item_timediffs[i]).to(device).unsqueeze(1)


                    ### These two things are being ignored, simply because our model does not require them right now
                    ### They are here, just in case they are needed in future
                    tbatch_itemids_previous = torch.LongTensor(lib.current_tbatches_previous_item[i]).to(device)
                    item_embedding_previous = item_embeddings[tbatch_itemids_previous, :]

                    # PROJECT USER EMBEDDING TO CURRENT TIME AND PREDICT ITEM EMBEDDING
                    user_embedding_input = user_embeddings[tbatch_userids, :]
                    op_embedding_input = user_embeddings[tbatch_opids, :]
                    item_embedding_input = item_embeddings[tbatch_itemids, :]

                    user_projected_embedding, predicted_item_embedding = model.forward(
                                                       user_embedding_input,
                                                       op_embedding_input,
                                                       item_embedding_input, 
                                                       feature_tensor, 
                                                       user_timediffs_tensor, perform='predict')


                    # CALCULATE PREDICTION LOSS AND USER EMBEDDING LOSSES
                    loss += MSELoss(predicted_item_embedding, item_embedding_input)
                    loss += MSELoss(user_projected_embedding, op_embedding_input)

                    # UPDATE DYNAMIC EMBEDDINGS AFTER INTERACTION
                    user_embedding_output = model.forward(
                                               user_embedding_input,
                                               op_embedding_input,
                                               item_embedding_input, 
                                               feature_tensor, 
                                               user_timediffs_tensor, perform='interacting_user')

                    op_embedding_output = model.forward(
                                               op_embedding_input,
                                                user_embedding_input,
                                               item_embedding_input, 
                                               feature_tensor, 
                                               user_timediffs_tensor, perform='interacted_user')

                    item_embedding_output = model.forward(
                                               user_embedding_input,
                                               op_embedding_input,
                                               item_embedding_input, 
                                               feature_tensor, 
                                               user_timediffs_tensor, perform='item')

                    item_embeddings[tbatch_itemids,:] = item_embedding_output
                    user_embeddings[tbatch_userids,:] = user_embedding_output
                    user_embeddings[tbatch_opids, :] = op_embedding_output

        #             user_embeddings_timeseries[tbatch_interactionids,:] = user_embedding_output
        #             item_embeddings_timeseries[tbatch_interactionids,:] = item_embedding_output

        #             # CALCULATE LOSS TO MAINTAIN TEMPORAL SMOOTHNESS
        #             loss += MSELoss(item_embedding_output, item_embedding_input.detach())
        #             loss += MSELoss(user_embedding_output, user_embedding_input.detach())


            # BACKPROPAGATE ERROR AFTER END OF T-BATCH
            total_loss += loss.item()
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            # RESET LOSS FOR NEXT T-BATCH
            loss = 0
            item_embeddings.detach_() # Detachment is needed to prevent double propagation of gradient
            user_embeddings.detach_()
    #         item_embeddings_timeseries.detach_() 
    #         user_embeddings_timeseries.detach_()

            # REINITIALIZE
            reinitialize_tbatches(lib)
            tbatch_to_insert = -1
print("\n\nTotal loss in this epoch = %f" % (total_loss))


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [11]:
print(total_loss)

nan
